In [1]:
import os
import pathlib
import shutil
import json
import yaml
import locale
import argparse
import cv2

import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
# Download the git repository

#!git clone https://github.com/WongKinYiu/yolov9
%cd yolov9
!pip install -r requirements.txt -q

c:\Mansura\UTI-Revision\yolov9


In [11]:
!pip install torch torchvision torchaudio


In [3]:
import torch
import torchvision
import torchaudio

print("torch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)
print("torchaudio version:", torchaudio.__version__)


torch version: 2.2.0+cu118
torchvision version: 0.17.0+cu118
torchaudio version: 2.2.0+cu118


In [4]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [5]:
import urllib.request

# Download yolov9-c.pt
url_gelan_c = "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt"
file_name_gelan_c = "yolov9-c.pt"
urllib.request.urlretrieve(url_gelan_c, file_name_gelan_c)

# Download yolov9e-e.pt
url_gelan_e = "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt"
file_name_gelan_e = "yolov9-e.pt"
urllib.request.urlretrieve(url_gelan_e, file_name_gelan_e)


('yolov9-e.pt', <http.client.HTTPMessage at 0x20817b98cd0>)

In [3]:
# @title Model training config
dataset_path = "C:/Mansura/UTI-Revision/Dataset/Partial-Data/JET-G-Data"           # @param {type:"string"}
experiment_name = "training.1"              # @param {type:"string"}
model_name = "yolov9-e"  # @param ["yolov9-c", "yolov9-e"]
result_dir = "C:/Mansura/UTI-Revision/YOLOv9e/On-Partial-Data/JET-G-Data" # @param {type:"string"}
num_epochs = 100        # @param {type:"integer"}
epoch_patience = 25     # @param{type: "integer"}
optimizer = "SGD"       # @param ["SGD", "Adam", "AdamW", "LION"]
image_size = 640        # @param{type: "integer"}]
learning_rate = 0.01    # @param {type: "number"}
batch_size = 8         # @param {type: "integer"}
conf_threshold = 0.001  # @param {type: "number"}
iou_threshold = 0.5     # @param {type: "number"}
pretrained = True       # @param {type:"boolean"}

weight_name = f"{model_name}.pt"
cfg_name = f"models/detect/{model_name}.yaml"

In [4]:
# Setup dataset configuration

data_config = {
    "path": dataset_path,
    "train": "train",       # training data folder name
    "val": "val",           # validation data folder name
    "test": "test",         # testing data folder name

    # label to class name mappings
    "names": {
        0: "cast",
        1: "cryst",
        2: "epith",
        3: "epithn",
        4: "eryth",
        5: "leuko",
        6: "mycete"
    }
}

with open("data.yaml", "w") as f:
    yaml.dump(data_config, f)

In [5]:
# Start training

%run train_dual.py \
--batch-size {batch_size} \
--epochs {num_epochs} \
--img {image_size} \
--device 0 \
--min-items 0 \
--data data.yaml \
--weights {model_name}.pt \
--cfg models/detect/{model_name}.yaml \
--hyp hyp.scratch-high.yaml \
--optimizer {optimizer} \
--project {result_dir} \
--name {experiment_name} \
--patience {epoch_patience} \
--exist-ok 


train_dual: weights=yolov9-e.pt, cfg=models/detect/yolov9-e.yaml, data=data.yaml, hyp=hyp.scratch-high.yaml, epochs=100, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=C:/Mansura/UTI-Revision/YOLOv9e/On-Partial-Data/JET-G-Data, name=training.1, exist_ok=True, quad=False, cos_lr=False, flat_cos_lr=False, fixed_lr=False, label_smoothing=0.0, patience=25, freeze=[0], save_period=-1, seed=0, local_rank=-1, min_items=0, close_mosaic=0, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLO  v0.1-104-g5b1ea9a Python-3.11.5 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, cls_pw=1.0, obj=0.7, 

In [9]:
# Test and generate predictions

!python val_dual.py --weights "C:/Mansura/UTI-Revision/YOLOv9e/On-Partial-Data/JET-G-Data/training.1/weights/best.pt" \
--data data.yaml \
--save-txt \
--save-conf \
--imgsz {image_size} \
--batch {batch_size} \
--conf 0.01 \
--iou {iou_threshold} \
--device 0 \
--project {result_dir} \
--name {"testing.1"} \
--task test \
--exist-ok


val_dual: data=data.yaml, weights=['C:/Mansura/UTI-Revision/YOLOv9e/On-Partial-Data/JET-G-Data/training.1/weights/best.pt'], batch_size=8, imgsz=640, conf_thres=0.01, iou_thres=0.5, max_det=300, task=test, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=C:/Mansura/UTI-Revision/YOLOv9e/On-Partial-Data/JET-G-Data, name=testing.1, exist_ok=True, half=False, dnn=False, min_items=0
WARNING  confidence threshold 0.01 > 0.001 produces invalid results
YOLO  v0.1-104-g5b1ea9a Python-3.11.5 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

Fusing layers... 
yolov9-e summary: 839 layers, 68557066 parameters, 0 gradients, 240.7 GFLOPs

test: Scanning C:\Mansura\UTI-Revision\Dataset\Partial-Data\JET-G-Data\test\labels...:   0%|          | 0/200 00:00
test: Scanning C:\Mansura\UTI-Revision\Dataset\Partial-Data\JET-G-Data\test\labels... 1 images, 0 backgrounds, 0 corrupt:   0%|          | 1/200

In [8]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Define class names and colors
class_names = ['cast', 'cryst', 'epith', 'epithn', 'eryth', 'leuko', 'mycete']  # Replace with your class names
class_colors = ['r', 'g', 'b']  # Replace with your desired colors

# Function to draw bounding boxes on image
def draw_boxes(image_path, predicted_boxes, ground_truth_boxes, output_path):
    # Load test image
    test_image = plt.imread(image_path)

    fig, axes = plt.subplots(1, 2, figsize=(15, 10))  # Increase figure size


    # Plot test image with predicted bounding boxes
    axes[0].imshow(test_image)
    axes[0].set_title('Predicted Bounding Boxes', fontsize=17, pad=22)
    axes[0].tick_params(axis='both', which='major', labelsize=15)  # Increase the size of the tick labels
    for prediction in predicted_boxes:
        class_index, x_center, y_center, width, height, confidence = map(float, prediction.split())

        # Convert normalized coordinates to pixel coordinates
        img_width, img_height = test_image.shape[1], test_image.shape[0]
        x = int(x_center * img_width)
        y = int(y_center * img_height)
        w = int(width * img_width)
        h = int(height * img_height)

        # Get class name and color
        class_name = class_names[int(class_index)]
        color = class_colors[int(class_index) % len(class_colors)]

        # Draw bounding box
        rect = patches.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=5, edgecolor=color, facecolor='none')
        axes[0].add_patch(rect)

        # Add class name and confidence score with background color
        bbox_props = dict(boxstyle="round,pad=0.5", edgecolor=color, facecolor="yellow", alpha=0.8)
        axes[0].text(x - w / 2, y - h / 2 - 10, f'{class_name}: {confidence:.2f}', color='black', fontsize=14, fontweight='bold', bbox=bbox_props)

    # Plot test image with ground truth bounding boxes
    axes[1].imshow(test_image)
    axes[1].set_title('Ground Truth Bounding Boxes', fontsize=17, pad=22)
    axes[1].tick_params(axis='both', which='major', labelsize=15)  # Increase the size of the tick labels
    for ground_truth in ground_truth_boxes:
        if not ground_truth or len(ground_truth.split()) != 5:
            continue  # Skip empty or improperly formatted ground truth bounding box lines

        class_index, x_center, y_center, width, height = map(float, ground_truth.split())

        # Convert normalized coordinates to pixel coordinates
        x = int(x_center * img_width)
        y = int(y_center * img_height)
        w = int(width * img_width)
        h = int(height * img_height)

        # Get class name and color
        class_name = class_names[int(class_index)]
        color = class_colors[int(class_index) % len(class_colors)]

        # Draw bounding box
        rect = patches.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=5, edgecolor=color, facecolor='none')
        axes[1].add_patch(rect)

        # Add class name with background color
        bbox_props = dict(boxstyle="round,pad=0.5", edgecolor=color, facecolor="yellow", alpha=0.8)
        axes[1].text(x - w / 2, y - h / 2 - 10, class_name, color='black', fontsize=14, fontweight='bold', bbox=bbox_props)

    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

# Path to input folders containing images and predicted labels
images_folder = 'C:/Mansura/UTI/data/DATA-UTI-LR/Data/test/images'  # Replace with your images folder path
predicted_labels_folder = 'C:/Mansura/UTI/Version-7/YOLOv9-e/testing.1-6/labels'  # Replace with your predicted labels folder path

# Path to folder containing ground truth labels
ground_truth_labels_folder = 'C:/Mansura/UTI/data/DATA-UTI-LR/Data/test/labels'  # Replace with your ground truth labels folder path

# Path to output folder to save annotated images
output_folder = 'C:/Mansura/UTI/Version-7/YOLOv9-e/testing.1-6/Prediction'

# Iterate over image files in the images folder
for image_file in os.listdir(images_folder):
    if image_file.endswith('.jpg'):
        # Construct paths for the image, predicted label file, and corresponding ground truth label file
        image_path = os.path.join(images_folder, image_file)
        predicted_label_path = os.path.join(predicted_labels_folder, image_file.replace('.jpg', '.txt'))
        ground_truth_label_path = os.path.join(ground_truth_labels_folder, image_file.replace('.jpg', '.txt'))

        # Check if the predicted label file exists
        if not os.path.exists(predicted_label_path):
            print(f"Predicted label file not found for image: {image_file}. Skipping...")
            continue

        # Check if the ground truth label file exists
        if not os.path.exists(ground_truth_label_path):
            print(f"Ground truth label file not found for image: {image_file}. Skipping...")
            continue

        # Load predicted bounding boxes from label text file
        with open(predicted_label_path, 'r') as f:
            predicted_lines = f.readlines()

        # Load ground truth bounding boxes from label text file
        with open(ground_truth_label_path, 'r') as f:
            ground_truth_lines = f.readlines()

        # Construct output path for annotated image
        output_filename = image_file.replace('.jpg', '_annotated.jpg')
        output_path = os.path.join(output_folder, output_filename)

        # Draw bounding boxes on test image and save it
        draw_boxes(image_path, predicted_lines, ground_truth_lines, output_path)


Predicted label file not found for image: nh00606.jpg. Skipping...


Predicted label to predicted image, no ground truth side by side

In [14]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Define class names and colors
class_names = ['cast', 'cryst', 'epith', 'epithn', 'eryth', 'leuko', 'mycete']  # Replace with your class names
class_colors = ['r', 'g', 'b']  # Replace with your desired colors

# Function to draw bounding boxes on image
def draw_boxes(image_path, predicted_boxes, output_path):
    # Load test image
    test_image = plt.imread(image_path)

    fig, ax = plt.subplots(figsize=(15, 10))  # Increase figure size

    # Plot test image with predicted bounding boxes
    ax.imshow(test_image)
    ax.set_title('Predicted Bounding Boxes', fontsize=17, pad=22)
    ax.tick_params(axis='both', which='major', labelsize=15)  # Increase the size of the tick labels

    for prediction in predicted_boxes:
        class_index, x_center, y_center, width, height, confidence = map(float, prediction.split())

        # Convert normalized coordinates to pixel coordinates
        img_width, img_height = test_image.shape[1], test_image.shape[0]
        x = int(x_center * img_width)
        y = int(y_center * img_height)
        w = int(width * img_width)
        h = int(height * img_height)

        # Get class name and color
        class_name = class_names[int(class_index)]
        color = class_colors[int(class_index) % len(class_colors)]

        # Draw bounding box
        rect = patches.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=5, edgecolor=color, facecolor='none')
        ax.add_patch(rect)

        # Add class name and confidence score with background color
        bbox_props = dict(boxstyle="round,pad=0.5", edgecolor=color, facecolor="yellow", alpha=0.8)
        ax.text(x - w / 2, y - h / 2 - 10, f'{class_name}: {confidence:.2f}', color='black', fontsize=14, fontweight='bold', bbox=bbox_props)

    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

# Path to input folders containing images and predicted labels
images_folder = 'C:/Mansura/UTI/data/External-Validation/A-clinical1/test/images'  # Replace with your images folder path
predicted_labels_folder = 'C:/Mansura/UTI/data/External-Validation/A-clinical1/test/labels'  # Replace with your predicted labels folder path

# Path to output folder to save annotated images
output_folder = 'C:/Mansura/UTI/FinalQualitativeResult/EV-GroundTruth'

# Iterate over image files in the images folder
for image_file in os.listdir(images_folder):
    if image_file.endswith('.jpg'):
        # Construct paths for the image and predicted label file
        image_path = os.path.join(images_folder, image_file)
        predicted_label_path = os.path.join(predicted_labels_folder, image_file.replace('.jpg', '.txt'))

        # Check if the predicted label file exists
        if not os.path.exists(predicted_label_path):
            print(f"Predicted label file not found for image: {image_file}. Skipping...")
            continue

        # Load predicted bounding boxes from label text file
        with open(predicted_label_path, 'r') as f:
            predicted_lines = f.readlines()

        # Construct output path for annotated image
        output_filename = image_file.replace('.jpg', '_annotated.jpg')
        output_path = os.path.join(output_folder, output_filename)

        # Draw bounding boxes on test image and save it
        draw_boxes(image_path, predicted_lines, output_path)


Applying TTA

In [8]:
# Test and generate predictions

!python val_dual.py --weights {result_dir}/{experiment_name}/weights/best.pt \
--data data.yaml \
--save-txt \
--save-conf \
--imgsz {image_size} \
--augment \
--batch {batch_size} \
--conf 0.01 \
--iou {iou_threshold} \
--device 0 \
--project {result_dir} \
--name {"A-testing.1"} \
--task test \
--exist-ok


val_dual: data=data.yaml, weights=['C:/Mansura/UTI/Version-7/YOLOv9-e/training.1/weights/best.pt'], batch_size=8, imgsz=640, conf_thres=0.01, iou_thres=0.5, max_det=300, task=test, device=0, workers=8, single_cls=False, augment=True, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=C:/Mansura/UTI/Version-7/YOLOv9-e, name=A-testing.1, exist_ok=True, half=False, dnn=False, min_items=0
WARNING  confidence threshold 0.01 > 0.001 produces invalid results
YOLO  v0.1-102-g01c0fe1 Python-3.11.5 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

Fusing layers... 
yolov9-e summary: 839 layers, 68557066 parameters, 0 gradients, 240.7 GFLOPs

test: Scanning C:\Mansura\UTI\data\DATA-UTI-LR\Data\test\labels...:   0%|          | 0/852 00:00
test: Scanning C:\Mansura\UTI\data\DATA-UTI-LR\Data\test\labels... 1 images, 0 backgrounds, 0 corrupt:   0%|          | 1/852 00:03
test: Scanning C:\Mansura\UTI\data\DATA-UTI-LR\Data\test\labels... 524 images, 0

In [9]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Define class names and colors
class_names = ['cast', 'cryst', 'epith', 'epithn', 'eryth', 'leuko', 'mycete']  # Replace with your class names
class_colors = ['r', 'g', 'b']  # Replace with your desired colors

# Function to draw bounding boxes on image
def draw_boxes(image_path, predicted_boxes, ground_truth_boxes, output_path):
    # Load test image
    test_image = plt.imread(image_path)

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Plot test image with predicted bounding boxes
    axes[0].imshow(test_image)
    axes[0].set_title('Predicted Bounding Boxes')
    for prediction in predicted_boxes:
        class_index, x_center, y_center, width, height, confidence = map(float, prediction.split())

        # Convert normalized coordinates to pixel coordinates
        img_width, img_height = test_image.shape[1], test_image.shape[0]
        x = int(x_center * img_width)
        y = int(y_center * img_height)
        w = int(width * img_width)
        h = int(height * img_height)

        # Get class name and color
        class_name = class_names[int(class_index)]
        color = class_colors[int(class_index) % len(class_colors)]

        # Draw bounding box
        rect = patches.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=1, edgecolor=color, facecolor='none')
        axes[0].add_patch(rect)

        # Add class name and confidence score
        axes[0].text(x - w / 2, y - h / 2 - 5, f'{class_name}: {confidence:.2f}', color=color)

    # Plot test image with ground truth bounding boxes
    axes[1].imshow(test_image)
    axes[1].set_title('Ground Truth Bounding Boxes')
    for ground_truth in ground_truth_boxes:
        class_index, x_center, y_center, width, height = map(float, ground_truth.split())

        # Convert normalized coordinates to pixel coordinates
        x = int(x_center * img_width)
        y = int(y_center * img_height)
        w = int(width * img_width)
        h = int(height * img_height)

        # Get class name and color
        class_name = class_names[int(class_index)]
        color = class_colors[int(class_index) % len(class_colors)]

        # Draw bounding box
        rect = patches.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=1, edgecolor=color, facecolor='none')
        axes[1].add_patch(rect)

        # Add class name
        axes[1].text(x - w / 2, y - h / 2 - 5, class_name, color=color)

    plt.savefig(output_path)
    plt.close()

# Path to input folders containing images and predicted labels
images_folder = 'C:/Mansura/UTI/Version-2/data/test/images'  # Replace with your images folder path
predicted_labels_folder = 'C:/Mansura/UTI/Version-2/gelan-e/testing.1/labels'  # Replace with your predicted labels folder path

# Path to folder containing ground truth labels
ground_truth_labels_folder = 'C:/Mansura/UTI/Version-2/data/test/labels'  # Replace with your ground truth labels folder path

# Path to output folder to save annotated images
output_folder = 'C:/Mansura/UTI/Version-2/gelan-e/testing.1/Prediction'  # Replace with your output folder path

# Iterate over image files in the images folder
for image_file in os.listdir(images_folder):
    if image_file.endswith('.jpg'):
        # Construct paths for the image, predicted label file, and corresponding ground truth label file
        image_path = os.path.join(images_folder, image_file)
        predicted_label_path = os.path.join(predicted_labels_folder, image_file.replace('.jpg', '.txt'))
        ground_truth_label_path = os.path.join(ground_truth_labels_folder, image_file.replace('.jpg', '.txt'))

        # Load predicted bounding boxes from label text file
        with open(predicted_label_path, 'r') as f:
            predicted_lines = f.readlines()

        # Load ground truth bounding boxes from label text file
        with open(ground_truth_label_path, 'r') as f:
            ground_truth_lines = f.readlines()

        # Construct output path for annotated image
        output_filename = image_file.replace('.jpg', '_annotated.jpg')
        output_path = os.path.join(output_folder, output_filename)

        # Draw bounding boxes on test image and save it
        draw_boxes(image_path, predicted_lines, ground_truth_lines, output_path)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Mansura/UTI/Version-2/gelan-e/testing.1/labels\\nh02032_jpg.rf.01e525f7810111b4cfb2591a433f99f1.txt'

In [7]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Define class names and colors
class_names = ['cast', 'cryst', 'epith', 'epithn', 'eryth', 'leuko', 'mycete']  # Replace with your class names
class_colors = ['r', 'g', 'b']  # Replace with your desired colors

# Function to draw bounding boxes on image
def draw_boxes(image_path, predicted_boxes, ground_truth_boxes, output_path):
    # Load test image
    test_image = plt.imread(image_path)

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Plot test image with predicted bounding boxes
    axes[0].imshow(test_image)
    axes[0].set_title('Predicted Bounding Boxes')
    for prediction in predicted_boxes:
        class_index, x_center, y_center, width, height, confidence = map(float, prediction.split())

        # Convert normalized coordinates to pixel coordinates
        img_width, img_height = test_image.shape[1], test_image.shape[0]
        x = int(x_center * img_width)
        y = int(y_center * img_height)
        w = int(width * img_width)
        h = int(height * img_height)

        # Get class name and color
        class_name = class_names[int(class_index)]
        color = class_colors[int(class_index) % len(class_colors)]

        # Draw bounding box
        rect = patches.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=1, edgecolor=color, facecolor='none')
        axes[0].add_patch(rect)

        # Add class name and confidence score
        axes[0].text(x - w / 2, y - h / 2 - 5, f'{class_name}: {confidence:.2f}', color=color)

    # Plot test image with ground truth bounding boxes
    axes[1].imshow(test_image)
    axes[1].set_title('Ground Truth Bounding Boxes')
    for ground_truth in ground_truth_boxes:
        class_index, x_center, y_center, width, height = map(float, ground_truth.split())

        # Convert normalized coordinates to pixel coordinates
        x = int(x_center * img_width)
        y = int(y_center * img_height)
        w = int(width * img_width)
        h = int(height * img_height)

        # Get class name and color
        class_name = class_names[int(class_index)]
        color = class_colors[int(class_index) % len(class_colors)]

        # Draw bounding box
        rect = patches.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=1, edgecolor=color, facecolor='none')
        axes[1].add_patch(rect)

        # Add class name
        axes[1].text(x - w / 2, y - h / 2 - 5, class_name, color=color)

    plt.savefig(output_path)
    plt.close()

# Path to input folders containing images and predicted labels
images_folder = 'C:/Mansura/content/uti-1/test/images'  # Replace with your images folder path
predicted_labels_folder = 'C:/Mansura/content/uti-1/v1.3-training.2-gelan-e/labels'  # Replace with your predicted labels folder path

# Path to folder containing ground truth labels
ground_truth_labels_folder = 'C:/Mansura/content/uti-1/test/labels'  # Replace with your ground truth labels folder path

# Path to output folder to save annotated images
output_folder = 'C:/Mansura/content/uti-1/v1.3-training.2-gelan-e/Prediction-0.2'  # Replace with your output folder path

# Iterate over image files in the images folder
for image_file in os.listdir(images_folder):
    if image_file.endswith('.jpg'):
        # Construct paths for the image, predicted label file, and corresponding ground truth label file
        image_path = os.path.join(images_folder, image_file)
        predicted_label_path = os.path.join(predicted_labels_folder, image_file.replace('.jpg', '.txt'))
        ground_truth_label_path = os.path.join(ground_truth_labels_folder, image_file.replace('.jpg', '.txt'))

        # Load predicted bounding boxes from label text file
        with open(predicted_label_path, 'r') as f:
            predicted_lines = f.readlines()

        # Load ground truth bounding boxes from label text file
        with open(ground_truth_label_path, 'r') as f:
            ground_truth_lines = f.readlines()

        # Construct output path for annotated image
        output_filename = image_file.replace('.jpg', '_annotated.jpg')
        output_path = os.path.join(output_folder, output_filename)

        # Draw bounding boxes on test image and save it
        draw_boxes(image_path, predicted_lines, ground_truth_lines, output_path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Mansura/content/uti-1/v1.3-training.2-gelan-e/labels\\nh02032_jpg.rf.01e525f7810111b4cfb2591a433f99f1.txt'

Gelan-c

In [16]:
# @title Model training config
dataset_path = "/content"           # @param {type:"string"}
experiment_name = "v1.2-gelan-c-adam"              # @param {type:"string"}
model_name = "gelan-c"  # @param ["gelan-c", "gelan-e"]
result_dir = "/content/drive/MyDrive/Mansura" # @param {type:"string"}
num_epochs = 100        # @param {type:"integer"}
epoch_patience = 20     # @param{type: "integer"}
optimizer = "SGD"       # @param ["SGD", "Adam", "AdamW", "LION"]
image_size = 640        # @param{type: "integer"}]
# learning_rate = 0.01    # @param {type: "number"}
batch_size = 8         # @param {type: "integer"}
conf_threshold = 0.001  # @param {type: "number"}
iou_threshold = 0.5     # @param {type: "number"}
pretrained = True       # @param {type:"boolean"}

weight_name = f"{model_name}.pt"
cfg_name = f"models/detect/{model_name}.yaml"

In [17]:
# Start training

!python train.py \
--batch-size {batch_size} \
--epochs {num_epochs} \
--img {image_size} \
--device 0 \
--min-items 0 \
--data data.yaml \
--weights {model_name}.pt \
--cfg models/detect/{model_name}.yaml \
--hyp hyp.scratch-high.yaml \
--optimizer {optimizer} \
--project {result_dir} \
--name {experiment_name} \
--patience {epoch_patience} \
--exist-ok


2024-04-18 15:31:27.359982: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 15:31:27.360029: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 15:31:27.361886: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 15:31:28.447076: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=gelan-c.pt, cfg=models/detect/gelan-c.yaml, data=data.yaml, hyp=hyp.scratch-high.yaml, epochs=100, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, b

In [18]:
# Test and generate predictions

!python val.py --weights {result_dir}/{experiment_name}/weights/best.pt \
--data data.yaml \
--save-txt \
--save-conf \
--imgsz {image_size} \
--batch {batch_size} \
--conf {conf_threshold} \
--iou {iou_threshold} \
--device 0 \
--project {result_dir} \
--name {experiment_name} \
--task test \
--exist-ok


val: data=data.yaml, weights=['/content/drive/MyDrive/Mansura/v1.2/weights/best.pt'], batch_size=8, imgsz=640, conf_thres=0.001, iou_thres=0.5, max_det=300, task=test, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=/content/drive/MyDrive/Mansura, name=v1.2, exist_ok=True, half=False, dnn=False, min_items=0
YOLO 🚀 v0.1-88-g03f920b Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

Fusing layers... 
gelan-c summary: 387 layers, 25232485 parameters, 0 gradients, 101.8 GFLOPs
test: Scanning /content/test/labels.cache... 523 images, 0 backgrounds, 0 corrupt: 100% 523/523 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances          P  